In [ ]:
import csv
from pandas import Series, DataFrame
import pandas as pd
import requests
import json
from os import name
import xml.etree.ElementTree as ET
import pandas as pd
from bs4 import BeautifulSoup
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote
import re
import numpy as np
import pandas as pd
import time
import pickle
import gzip
import pprint
from datetime import datetime
from datetime import timedelta
##API Key
serviceKey = 'Enter your API KEY'

In [ ]:
######Part 1-1 현재일자 기준 인천도착 미래 정기 운항 스케줄 호출(국내선, 국제선 포함)
import requests
from bs4 import BeautifulSoup
data=[]
url = 'https://apis.data.go.kr/B551177/StatusOfPaxFltSched/getPaxFltSchedArrivalsDeOdp'
params ={'serviceKey' : serviceKey, 'numOfRows' : 100 }
response = requests.get(url, params=params)
soup_inc = BeautifulSoup(response.text, 'html.parser')
data = []
for item in soup_inc.find_all('item'):
    item_data = {}
    for tag in item.find_all():
        item_data[tag.name] = tag.text
    data.append(item_data)

df_inc_in= pd.DataFrame(data)
##df_inc.head()


In [ ]:
###airport 체계가 한국공항공사와 달라서 확인 필요함 (ex: 보홀 팡라오)

,airline,airport,airportcode,codeshare,firstdate,flightid,friday,lastdate,masterflightid,monday,saturday,season,st,sunday,thursday,tuesday,wednesday
0,베트남항공,나트랑,CXR,Master,20231029,VN440,Y,20240330,,Y,Y,W23,0425,Y,Y,Y,Y
1,대한항공,나트랑,CXR,Slave,20231029,KE5680,Y,20240330,VN440,Y,Y,W23,0425,Y,Y,Y,Y
2,베트남항공,하노이,HAN,Master,20231029,VN416,Y,20240330,,Y,Y,W23,0530,Y,Y,Y,Y
3,대한항공,하노이,HAN,Slave,20231029,KE5684,Y,20240330,VN416,Y,Y,W23,0530,Y,Y,Y,Y
4,필리핀항공,마닐라,MNL,Master,20231029,PR466,Y,20240330,,Y,Y,W23,0600,Y,Y,Y,Y


In [ ]:
######Part 1-2 현재일자 기준 인천 출발하는 항공편
##미래 데이터 적재를 위한 loop
#현재 날짜 가져옴
##today = datetime.today()
today = datetime.today()
data_inc_to_load = []

#미래 30일 데이터
for _ in range(30):
    for index, row in df_inc_in.iterrows():
        # 날짜 포맷을 고려해 날짜 데이터를 가져옴
        start_date = datetime.strptime(row['firstdate'], '%Y%m%d')
        end_date = datetime.strptime(row['lastdate'], '%Y%m%d')

        # 오늘의 요일을 가져와 대응하는 열을 확인
        today_weekday = today.strftime('%A').lower()

        # 오늘이 시작일자와 종료일자 사이에 있고 해당 요일에 운행하는 경우 데이터를 삽입
        if start_date <= today <= end_date and row[today_weekday] == 'Y':
            row['date'] = today.date()
            data_inc_to_load.append(row)

    # 다음 날짜로 이동
    today += timedelta(days=1)

# 적재된 데이터 출력
df_inc_in = pd.DataFrame(data_inc_to_load)
##result_df.to_csv('test.csv', header=True, encoding='cp949')
df_inc_in.head()


,airline,airport,airportcode,codeshare,firstdate,flightid,friday,lastdate,masterflightid,monday,saturday,season,st,sunday,thursday,tuesday,wednesday,date
0,베트남항공,나트랑,CXR,Master,20231029,VN440,Y,20240330,,Y,Y,W23,0425,Y,Y,Y,Y,2023-11-08
1,대한항공,나트랑,CXR,Slave,20231029,KE5680,Y,20240330,VN440,Y,Y,W23,0425,Y,Y,Y,Y,2023-11-08
2,베트남항공,하노이,HAN,Master,20231029,VN416,Y,20240330,,Y,Y,W23,0530,Y,Y,Y,Y,2023-11-08
3,대한항공,하노이,HAN,Slave,20231029,KE5684,Y,20240330,VN416,Y,Y,W23,0530,Y,Y,Y,Y,2023-11-08
4,필리핀항공,마닐라,MNL,Master,20231029,PR466,Y,20240330,,Y,Y,W23,0600,Y,Y,Y,Y,2023-11-08


In [ ]:
#####Part 1-3 데이터 프레임 전처리 및 현재 테이블 레이아웃에 맞게 변형하기 위한 함수 정의 및 국가/대륙 붙이기 위한 파일 import
##공항과 도시명 변환하기 위한 함수
def split_city_and_airport(row) :
    ##city_parts = row['city'].split('/')
    airport_parts = row['airport'].split('/')
    ##if len(city_parts)>1 :
    ##    row['city'] = city_parts[1]
    if len(airport_parts)>1 :
        row['airport'] = airport_parts[1]
    return row


country = pd.read_csv('airport-codes_csv.csv', keep_default_na=False)
country = country[(country.iata_code.notnull()) & (country.type != 'closed') & (country.type != 'heliport')].loc[:,['continent', 'iso_country', 'iata_code']]
country = country.drop_duplicates(keep='first').loc[country.drop_duplicates(keep='first').iata_code.drop_duplicates(keep=False).index, :]

In [ ]:
#####Part 1-4 해외에서 도착하는 Outbound 편(target : tb_ex_flght_arvplc_gp)
#해외에서 도착하는 Inbound 항공편이기 때문에 도착지(citycode) 기준으로 join
df_inc_in = pd.merge(df_inc_in, country, how = 'left', left_on='airportcode', right_on='iata_code' )

##Outbound는 airport, airportcode가 출발지, city, citycode가 도착지
df_inc_in = df_inc_in.apply(split_city_and_airport, axis=1)  #공항과 도착지 변환 함수 적용

df_inc_in_fnl = pd.DataFrame()

##미래 데이터 API에서는 estimated_time 필드와 actual_time 필드는 그냥 :로 처리해 줄 수 밖에 없음
df_inc_in_fnl['airline'] = df_inc_in['airline']
df_inc_in_fnl['flight_no'] = df_inc_in['flightid']
df_inc_in_fnl['starting_point'] = df_inc_in['airportcode'] + '('  + df_inc_in['airport'] + ')'
df_inc_in_fnl['planned_time'] = df_inc_in['st'].str[:2] + ':' + df_inc_in['st'].str[2:]      ##hhmm 으로 있어서 hh:mm으로 변환 필요
df_inc_in_fnl['estimated_time'] = ':'
df_inc_in_fnl['actual_time'] = ':'
df_inc_in_fnl['type']  = '여객'   ###화물 확인하고 추가 변환 해줘야함!!!! 인천 데이터에는 화물 여객 구분 없음
df_inc_in_fnl['status'] = '출발예정'
df_inc_in_fnl['search_date'] = df_inc_in['date']
df_inc_in_fnl['airport'] = '인천'
df_inc_in_fnl['continent'] = df_inc_in['continent']
df_inc_in_fnl['iso_country'] = df_inc_in['iso_country']
df_inc_in_fnl['etl_tm'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


########### s3에 떨구고 redshift tb_ex_flght_arvplc 테이블에 적재하는 로직만 쓰면됨

df_inc_in_fnl.head()

,airline,flight_no,starting_point,planned_time,estimated_time,actual_time,type,status,search_date,airport,continent,iso_country,etl_tm
0,베트남항공,VN440,CXR(나트랑),04:25,:,:,여객,출발예정,2023-11-08,인천,AS,VN,2023-11-08 17:30:45
1,대한항공,KE5680,CXR(나트랑),04:25,:,:,여객,출발예정,2023-11-08,인천,AS,VN,2023-11-08 17:30:45
2,베트남항공,VN416,HAN(하노이),05:30,:,:,여객,출발예정,2023-11-08,인천,AS,VN,2023-11-08 17:30:45
3,대한항공,KE5684,HAN(하노이),05:30,:,:,여객,출발예정,2023-11-08,인천,AS,VN,2023-11-08 17:30:45
4,필리핀항공,PR466,MNL(마닐라),06:00,:,:,여객,출발예정,2023-11-08,인천,AS,PH,2023-11-08 17:30:45
